In [1]:
import pandas as pd
import numpy as np
from numpy.random import RandomState
import datetime
from pandas.tseries.holiday import USFederalHolidayCalendar
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, MinMaxScaler,StandardScaler
import tensorflow as tf
# tf.random.set_seed(2023)
from tensorflow import keras
from tensorflow.keras.layers import LSTM, Dense, Dropout, TimeDistributed, Lambda
from sklearn.model_selection import train_test_split
from keras.preprocessing.sequence import TimeseriesGenerator
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio

2023-03-09 21:20:02.542877: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Task 1: Predict time used in seconds per hour

### Data and Model Preperation

In [2]:
df = pd.read_csv("../data/processed/new_lstm_dataset_local.csv", parse_dates=['Start', 'End'])
df['Duration'] = df['Duration'].apply(lambda x: pd.Timedelta(x))
df['Time_diff_sec'] = df['Duration'].apply(lambda x: x.total_seconds())
df = df.drop(columns='Duration')
delta = datetime.timedelta(hours=1)
df['sec_to_next_hr'] = df['Start'].apply(lambda x: ((x+delta).replace(microsecond=0, second=0, minute=0) - x).seconds)

In [3]:
def helper(row):
    delta = datetime.timedelta(hours=1)
    if row['Time_diff_sec'] < row['sec_to_next_hr']:
        row['End'] = row['Start'] + pd.to_timedelta(row['Time_diff_sec'], unit='S')
        return [row]
    row2 = row.copy()
    
    row['End'] = (row['Start']+delta).floor('H')
    row2['Start'] = row['End']
    row2['End'] = (row['End']+delta).floor('H')
    
    row2['Time_diff_sec'] = row['Time_diff_sec'] - row['sec_to_next_hr']
    row2['sec_to_next_hr'] = 3600
    row['Time_diff_sec'] = row['sec_to_next_hr']
    return [row] + helper(row2)

def func(row):
    if row['Time_diff_sec'] > row['sec_to_next_hr']:
        return pd.DataFrame(helper(row))
    return pd.DataFrame([row])

In [4]:
processed_df = pd.concat([func(row) for _, row in df.iterrows()], ignore_index=True)

df_1 = processed_df[processed_df['Value'] == 'firefox.exe'].reset_index()
temp = df_1[['Start', 'Time_diff_sec']]
temp = temp.groupby(pd.Grouper(key='Start', freq='H'))['Time_diff_sec'].sum()
layout = go.Layout(
    title='Firefox used in seconds per hour',
#     xaxis={'title':'Date'},
    yaxis={'title':'Duration(s)'}
)
fig = go.Figure([go.Scatter(x=temp.index, y=temp.values)], layout=layout)
fig.show()

In [5]:
df_1 = processed_df[processed_df['Value'] == 'firefox.exe'].reset_index()
temp = df_1[['Start', 'Time_diff_sec']]
temp = temp.groupby(pd.Grouper(key='Start', freq='H'))['Time_diff_sec'].sum()
layout = go.Layout(
    title='Firefox used in seconds per hour',
#     xaxis={'title':'Date'},
    yaxis={'title':'Duration(s)'}
)
fig = go.Figure([go.Scatter(x=temp.index, y=temp.values)], layout=layout)
fig.update_xaxes(
        rangebreaks=[dict(values=pd.date_range('2022-12-23', '2023-01-08'))] # hide dates with no values
    )
fig.show()

#### For experiments, we keep our model structure the same

In [6]:
def train_model(X_train, y_train, num_epochs = 400, n_steps=1):
    feature_shape = X_train.shape[2]

    model = keras.Sequential()
    model.add(LSTM(32, return_sequences=True, input_shape=(n_steps, feature_shape)))
    # model.add(Dropout(0.2))

    # model.add(LSTM(32, return_sequences=True))
    # model.add(Dropout(0.2))

    model.add(LSTM(16, return_sequences=True))
    # model.add(Dropout(0.2))

    model.add(LSTM(16))
    # model.add(Dropout(0.2))

    # model.add(TimeDistributed(Dense(1)))
    model.add(Dense(32))
    model.add(Dense(16))
    model.add(Dense(1))
    opt = keras.optimizers.Adam(learning_rate=0.0001)
    model.compile(optimizer=opt, loss='mse')
    
    print(model.summary())

    history = model.fit(X_train, y_train,epochs=num_epochs, verbose=0)
    return model, history

In [7]:
def transformation(column, max_value):
#   max_value = column.max()
  sin_values = [np.sin((2*np.pi*x)/max_value) for x in list(column)]
  cos_values = [np.cos((2*np.pi*x)/max_value) for x in list(column)]
  return sin_values, cos_values

def process(temp, exp):
    if exp == 1:
        temp['dayofweek'] = temp['Start'].apply(lambda x: x.dayofweek)
        temp['dayofmonth'] = temp['Start'].apply(lambda x: x.day)
        temp['dayofyear'] = temp['Start'].apply(lambda x: x.dayofyear)
        temp['hour'] = temp['Start'].apply(lambda x: x.hour)
        # temp['minute'] = temp['Start'].apply(lambda x: x.minute)
        temp['month'] = temp['Start'].apply(lambda x: x.month)
        temp['is_weekend'] = temp['Start'].apply(lambda x: 1 if x == 5 or x == 6 else -1)
        temp['is_winter_holiday'] = temp['Start'].apply(lambda x: 1 if x > datetime.datetime(2022, 12, 12) or x < datetime.datetime(2023, 1, 8) else -1)
        data = temp.drop(columns='Start').values

    elif exp == 2 or exp == 4:
        temp['dayofweek'] = temp['Start'].apply(lambda x: x.dayofweek).astype('category')
        temp['dayofmonth'] = temp['Start'].apply(lambda x: x.day).astype('category')
        temp['dayofyear'] = temp['Start'].apply(lambda x: x.dayofyear).astype('category')
        temp['hour'] = temp['Start'].apply(lambda x: x.hour).astype('category')
        # temp['minute'] = temp['Start'].apply(lambda x: x.minute).astype('category')
        temp['month'] = temp['Start'].apply(lambda x: x.month).astype('category')
        temp['is_weekend'] = temp['Start'].apply(lambda x: 1 if x == 5 or x == 6 else -1)
        temp['is_winter_holiday'] = temp['Start'].apply(lambda x: 1 if x > datetime.datetime(2022, 12, 12) or x < datetime.datetime(2023, 1, 8) else -1)
        temp = temp.drop(columns='Start')
        # data = pd.get_dummies(temp, columns=['dayofweek', 'dayofmonth', 'dayofyear', 'hour','month']).values
        data = pd.get_dummies(temp, columns=['dayofweek', 'dayofmonth', 'dayofyear', 'hour','minute','month']).values
    
    elif exp == 3 or exp == 5:
        temp['dayofweek'] = temp['Start'].apply(lambda x: x.dayofweek)
        temp['dayofmonth'] = temp['Start'].apply(lambda x: x.day)
        temp['dayofyear'] = temp['Start'].apply(lambda x: x.dayofyear)
        temp['hour'] = temp['Start'].apply(lambda x: x.hour)
        # temp['minute'] = temp['Start'].apply(lambda x: x.minute)
        temp['month'] = temp['Start'].apply(lambda x: x.month)
        temp['is_weekend'] = temp['Start'].apply(lambda x: 1 if x == 5 or x == 6 else -1)
        temp['is_winter_holiday'] = temp['Start'].apply(lambda x: 1 if x > datetime.datetime(2022, 12, 12) or x < datetime.datetime(2023, 1, 8) else -1)
        # cols = [('dayofweek', 6), ('dayofmonth', 12), ('dayofyear', 366), ('hour', 23), ('month', 12)]
        cols = [('dayofweek', 6), ('dayofmonth', 12), ('dayofyear', 366), ('hour', 23), ('minute', 59), ('month', 12)]
        for col, max_value in cols:
            temp[f'{col}_sin'], temp[f'{col}_cos'] = transformation(temp[col], max_value)
        data = temp.drop(columns=['Start'] + [i[0] for i in cols]).values
    
    # display(temp)
    return data

In [54]:
def get_dataset(df, n_steps, start=None, end=None, exp=1):
    # temp = df.groupby(pd.Grouper(key='Start', freq='H')).sum().reset_index()
    temp = df.copy()
    temp['minute'] = temp['Start'].apply(lambda x: x.minute)
    temp = temp.groupby(pd.Grouper(key='Start', freq='H')).agg({
            'Time_diff_sec': 'sum',
            'minute': 'min'
        }).reset_index().fillna(0)
    temp = temp[(temp['Start'] < '2022-12-23') | (temp['Start'] > '2023-01-08')]

    scaler = MinMaxScaler()
    scaler.fit(temp[['Time_diff_sec']])
    temp[['Time_diff_sec']] = scaler.transform(temp[['Time_diff_sec']])

    data = process(temp, exp)
    
    X, y = [], []
    for i in range(len(data)-n_steps):
        # gather input and output parts of the pattern
        seq_x, seq_y = data[i:i+n_steps, :], data[i+n_steps, 0:1]
        X.append(seq_x)
        y.append(seq_y)
    
    X, y = np.array(X), np.array(y)
    if start is None:
        test_size = int(X.shape[0] * 0.2)
        start = np.random.randint(0, len(X)-test_size)
        end = start + test_size
    test_ind = np.zeros(X.shape[0], dtype=bool)
    test_ind[start:end] = True
    X_train, X_test = X[~test_ind, :, :], X[test_ind, :, :]
    y_train, y_test = y[~test_ind], y[test_ind]
    return X_train, y_train, X_test, y_test, scaler, start, end

In [9]:
def get_accuracy(pred, target, bound=10):
    # print(np.sum(np.abs(pred - target) < bound))
    return np.mean(np.abs(pred - target) < bound)

def get_nonzero_accuracy(pred, target, bound=10):
    ind = np.where(target!=0)[0]
    # print(np.sum(np.abs(pred[ind] - target[ind]) < bound))
    return np.mean(np.abs(pred[ind] - target[ind]) < bound)

In [10]:
def evaluate(model, X_train, y_train, X_test, y_test, train_pred, test_pred):

    train_loss = model.evaluate(X_train, y_train, verbose=0)
    train_acc_1 = get_accuracy(train_pred, y_train, 5)
    train_acc_2 = get_accuracy(train_pred, y_train, 10)
    train_acc_3 = get_accuracy(train_pred, y_train, 60)
    nonzero_train_acc_1 = get_nonzero_accuracy(train_pred, y_train, 5)
    nonzero_train_acc_2 = get_nonzero_accuracy(train_pred, y_train, 10)
    nonzero_train_acc_3 = get_nonzero_accuracy(train_pred, y_train, 60)
    test_loss = model.evaluate(X_test, y_test, verbose=0)
    test_acc_1 = get_accuracy(test_pred, y_test, 5)
    test_acc_2 = get_accuracy(test_pred, y_test, 10)
    test_acc_3 = get_accuracy(test_pred, y_test, 60)    
    nonzero_test_acc_1 = get_nonzero_accuracy(test_pred, y_test, 5)
    nonzero_test_acc_2 = get_nonzero_accuracy(test_pred, y_test, 10)
    nonzero_test_acc_3 = get_nonzero_accuracy(test_pred, y_test, 60)


    print(f'Train Loss: {train_loss}')
    print(f'Train Accuracy (abs diff within 5s): {train_acc_1}')
    print(f'Train Accuracy (abs diff within 10s): {train_acc_2}')
    print(f'Train Accuracy (abs diff within 60s): {train_acc_3}')
    print(f'Non-zero Train Accuracy (abs diff within 5s): {nonzero_train_acc_1}')
    print(f'Non-zero Train Accuracy (abs diff within 10s): {nonzero_train_acc_2}')
    print(f'Non-zero Train Accuracy (abs diff within 60s): {nonzero_train_acc_3}')
    print(f'Test Loss: {test_loss}')
    print(f'Test Accuracy (abs diff within 5s): {test_acc_1}')
    print(f'Test Accuracy (abs diff within 10s): {test_acc_2}')
    print(f'Test Accuracy (abs diff within 60s): {test_acc_3}')
    print(f'Non-zero Test Accuracy (abs diff within 5s): {nonzero_test_acc_1}')
    print(f'Non-zero Test Accuracy (abs diff within 10s): {nonzero_test_acc_2}')
    print(f'Non-zero Test Accuracy (abs diff within 60s): {nonzero_test_acc_3}')

def plot_prediction(df, train_pred, test_pred, n_steps, start, end):
        fig = make_subplots(rows=2, cols=1, subplot_titles=['Prediction and Ground Truth', "Training Loss"])
        layout = go.Layout(
            title='Firefox used in seconds per hour',
        #     xaxis={'title':'Date'},
            yaxis={'title':'Duration(s)'}
        )
        temp = df.groupby(pd.Grouper(key='Start', freq='H'))['Time_diff_sec'].sum().reset_index()
        temp = temp[(temp['Start'] < '2022-12-23') | (temp['Start'] > '2023-01-08')]
        temp = temp.iloc[n_steps:, :]
        dates = temp['Start'].values
        # test = dates[test_ind]
        # train1 = dates[:np.where(test_ind)[0][0]]
        # train2 = dates[np.where(test_ind)[0][-1]+1:]
        # assert len(train1)+len(train2) == len(train_pred)
        fig = go.Figure([
            go.Scatter(x=dates, y=temp['Time_diff_sec'].iloc[:].values, name='data'),
            go.Scatter(x=dates[:start], y=train_pred[:start,0], line=dict(color='red'),name='train prediction'),
            go.Scatter(x=dates[end:], y=train_pred[start:,0], line=dict(color='red'),name='train prediction', showlegend=False),
            go.Scatter(x=dates[start:end], y=test_pred[:,0], line=dict(color='#00CC96'),name='test prediction')
        ], layout=layout)
        fig.update_xaxes(
        rangebreaks=[dict(values=pd.date_range('2022-12-23', '2023-01-08'))] # hide dates with no values
    )
        fig.show()
        # pio.write_html(fig, '../../plots/experiment1')

def plot_loss(history):
        loss = history.history['loss']
        layout = go.Layout(
            xaxis={'title':'Epoch'},
            yaxis={'title':'Loss'}
        )
        fig = go.Figure([go.Scatter(x=list(range(400)), y=loss, mode='lines',name = 'Training Loss')], layout=layout)
        fig.show()


### Experiment 1: Vanilla Approach
- Input:
    - Numerical: Day of the Week, Day of the Month, Day of the Year, Hour, Minute, Date, Month
    - Binary: is Weekend, is Winter Holiday
- Activation Function:
    - LSTM: tanh
    - Dense: None(linear)
- Loss: 
    - Mean Squared Error
- Optimizer: 
    - Adam
- Learning_rate: 
    - 0.001
- Num_epoch: 
    - 400

In [11]:
n_steps = 5
X_train, y_train, X_test, y_test, scaler, start, end = get_dataset(df_1, n_steps)
X_train[0]

array([[ 3.11702222e-01,  3.40000000e+01,  0.00000000e+00,
         1.20000000e+01,  3.46000000e+02,  1.20000000e+01,
         1.20000000e+01, -1.00000000e+00,  1.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.20000000e+01,  3.46000000e+02,  1.30000000e+01,
         1.20000000e+01, -1.00000000e+00,  1.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.20000000e+01,  3.46000000e+02,  1.40000000e+01,
         1.20000000e+01, -1.00000000e+00,  1.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.20000000e+01,  3.46000000e+02,  1.50000000e+01,
         1.20000000e+01, -1.00000000e+00,  1.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.20000000e+01,  3.46000000e+02,  1.60000000e+01,
         1.20000000e+01, -1.00000000e+00,  1.00000000e+00]])

In [12]:
n_steps = 5
X_train, y_train, X_test, y_test, scaler, start, end = get_dataset(df_1, n_steps)

model, history = train_model(X_train, y_train, n_steps=n_steps)
train_pred = scaler.inverse_transform(model.predict(X_train, verbose=0))
test_pred = scaler.inverse_transform(model.predict(X_test, verbose=0))

2023-03-09 21:20:16.798018: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 5, 32)             5376      
                                                                 
 lstm_1 (LSTM)               (None, 5, 16)             3136      
                                                                 
 lstm_2 (LSTM)               (None, 16)                2112      
                                                                 
 dense (Dense)               (None, 32)                544       
                                                                 
 dense_1 (Dense)             (None, 16)                528       
                                                                 
 dense_2 (Dense)             (None, 1)                 17        
                                                                 
Total params: 11,713
Trainable params: 11,713
Non-traina

In [13]:
plot_prediction(df_1, train_pred, test_pred, n_steps, start, end)

In [14]:
plot_loss(history)

In [15]:
evaluate(model, X_train, y_train, X_test, y_test, train_pred, test_pred)

Train Loss: 0.012442393228411674
Train Accuracy (abs diff within 5s): 0.02987012987012987
Train Accuracy (abs diff within 10s): 0.06623376623376623
Train Accuracy (abs diff within 60s): 0.4090909090909091
Non-zero Train Accuracy (abs diff within 5s): 0.00641025641025641
Non-zero Train Accuracy (abs diff within 10s): 0.01282051282051282
Non-zero Train Accuracy (abs diff within 60s): 0.11538461538461539
Test Loss: 0.040179599076509476
Test Accuracy (abs diff within 5s): 0.03125
Test Accuracy (abs diff within 10s): 0.052083333333333336
Test Accuracy (abs diff within 60s): 0.2604166666666667
Non-zero Test Accuracy (abs diff within 5s): 0.0
Non-zero Test Accuracy (abs diff within 10s): 0.0
Non-zero Test Accuracy (abs diff within 60s): 0.05263157894736842


### Experiment 2: One Hot Encoded Approach
- Input:
    - **One-hot-encoded**: Day of the Week, Day of the Month, Day of the Year, Hour, Minute, Date, Month
    - Binary: is Weekend, is Winter Holiday
- Activation Function:
    - LSTM: tanh
    - Dense: None(linear)
- Loss: 
    - Mean Squared Error
- Optimizer: 
    - Adam
- Learning_rate: 
    - 0.001
- Num_epoch: 
    - 400

In [35]:
n_steps = 5
X_train, y_train, X_test, y_test, scaler, start, end = get_dataset(df_1, n_steps, start, end, 2)

model, history = train_model(X_train, y_train, n_steps=n_steps)
train_pred = scaler.inverse_transform(model.predict(X_train, verbose=0))
test_pred = scaler.inverse_transform(model.predict(X_test, verbose=0))

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_18 (LSTM)              (None, 5, 32)             23808     
                                                                 
 lstm_19 (LSTM)              (None, 5, 16)             3136      
                                                                 
 lstm_20 (LSTM)              (None, 16)                2112      
                                                                 
 dense_18 (Dense)            (None, 32)                544       
                                                                 
 dense_19 (Dense)            (None, 16)                528       
                                                                 
 dense_20 (Dense)            (None, 1)                 17        
                                                                 
Total params: 30,145
Trainable params: 30,145
Non-trai

In [36]:
plot_prediction(df_1, train_pred, test_pred, n_steps, start, end)

In [37]:
plot_loss(history)

In [38]:
evaluate(model, X_train, y_train, X_test, y_test, train_pred, test_pred)

Train Loss: 0.0021955466363579035
Train Accuracy (abs diff within 5s): 0.06753246753246753
Train Accuracy (abs diff within 10s): 0.12857142857142856
Train Accuracy (abs diff within 60s): 0.5584415584415584
Non-zero Train Accuracy (abs diff within 5s): 0.0
Non-zero Train Accuracy (abs diff within 10s): 0.00641025641025641
Non-zero Train Accuracy (abs diff within 60s): 0.12179487179487179
Test Loss: 0.06883930414915085
Test Accuracy (abs diff within 5s): 0.005208333333333333
Test Accuracy (abs diff within 10s): 0.026041666666666668
Test Accuracy (abs diff within 60s): 0.19791666666666666
Non-zero Test Accuracy (abs diff within 5s): 0.0
Non-zero Test Accuracy (abs diff within 10s): 0.0
Non-zero Test Accuracy (abs diff within 60s): 0.17543859649122806


### Experiment 3: Different Feature Engineering Approach
- Input:
    - **Take sin and cos on**: Day of the Week, Day of the Month, Day of the Year, Hour, Minute, Date, Month
    - Binary: is Weekend, is Winter Holiday
- Activation Function:
    - LSTM: tanh
    - Dense: None(linear)
- Loss: 
    - Mean Squared Error
- Optimizer: 
    - Adam
- Learning_rate: 
    - 0.001
- Num_epoch: 
    - 400

In [20]:
n_steps = 5
X_train, y_train, X_test, y_test, scaler, start, end = get_dataset(df_1, n_steps, start, end, 3)

model, history = train_model(X_train, y_train, n_steps=n_steps)
train_pred = scaler.inverse_transform(model.predict(X_train, verbose=0))
test_pred = scaler.inverse_transform(model.predict(X_test, verbose=0))

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_6 (LSTM)               (None, 5, 32)             6144      
                                                                 
 lstm_7 (LSTM)               (None, 5, 16)             3136      
                                                                 
 lstm_8 (LSTM)               (None, 16)                2112      
                                                                 
 dense_6 (Dense)             (None, 32)                544       
                                                                 
 dense_7 (Dense)             (None, 16)                528       
                                                                 
 dense_8 (Dense)             (None, 1)                 17        
                                                                 
Total params: 12,481
Trainable params: 12,481
Non-trai

In [21]:
plot_prediction(df_1, train_pred, test_pred, n_steps, start, end)

In [22]:
plot_loss(history)

In [23]:
evaluate(model, X_train, y_train, X_test, y_test, train_pred, test_pred)

Train Loss: 0.018502580001950264
Train Accuracy (abs diff within 5s): 0.024675324675324677
Train Accuracy (abs diff within 10s): 0.053246753246753244
Train Accuracy (abs diff within 60s): 0.3038961038961039
Non-zero Train Accuracy (abs diff within 5s): 0.01282051282051282
Non-zero Train Accuracy (abs diff within 10s): 0.01282051282051282
Non-zero Train Accuracy (abs diff within 60s): 0.04487179487179487
Test Loss: 0.043127816170454025
Test Accuracy (abs diff within 5s): 0.015625
Test Accuracy (abs diff within 10s): 0.020833333333333332
Test Accuracy (abs diff within 60s): 0.11458333333333333
Non-zero Test Accuracy (abs diff within 5s): 0.03508771929824561
Non-zero Test Accuracy (abs diff within 10s): 0.03508771929824561
Non-zero Test Accuracy (abs diff within 60s): 0.08771929824561403


### Experiment 4.1: Different activation function (sigmoid) for experiment 2
- Input:
    - One-hot-encoded: Day of the Week, Day of the Month, Day of the Year, Hour, Minute, Date, Month
    - Binary: is Weekend, is Winter Holiday
- Activation Function:
    - LSTM: tanh
    - Dense: **sigmoid**
- Loss: 
    - Mean Squared Error
- Optimizer: 
    - Adam
- Learning_rate: 
    - 0.001
- Num_epoch: 
    - 400

In [51]:
def train_model_sig(X_train, y_train, num_epochs = 400, n_steps=1):
    feature_shape = X_train.shape[2]

    model = keras.Sequential()
    model.add(LSTM(32, return_sequences=True, input_shape=(n_steps, feature_shape)))
    # model.add(Dropout(0.2))

    # model.add(LSTM(32, return_sequences=True))
    # model.add(Dropout(0.2))

    model.add(LSTM(16, return_sequences=True))
    # model.add(Dropout(0.2))

    model.add(LSTM(16))
    # model.add(Dropout(0.2))

    # model.add(TimeDistributed(Dense(1)))
    model.add(Dense(32))
    model.add(Dense(16))
    model.add(Dense(1, activation='sigmoid'))
    # model.add(Lambda(get_range))
    opt = keras.optimizers.Adam(learning_rate=0.0001)
    model.compile(optimizer=opt, loss='mse')

    history = model.fit(X_train, y_train,epochs=num_epochs, verbose=0)
    return model, history

In [52]:
n_steps = 5
X_train, y_train, X_test, y_test, scaler, start, end = get_dataset(df_1, n_steps, start, end, 4)

model, history = train_model_sig(X_train, y_train, n_steps=n_steps)
train_pred = scaler.inverse_transform(model.predict(X_train, verbose=0))
test_pred = scaler.inverse_transform(model.predict(X_test, verbose=0))

In [53]:
plot_prediction(df_1, train_pred, test_pred, n_steps, start, end)

In [55]:
plot_loss(history)

In [56]:
evaluate(model, X_train, y_train, X_test, y_test, train_pred, test_pred)

Train Loss: 0.0039169746451079845
Train Accuracy (abs diff within 5s): 0.825974025974026
Train Accuracy (abs diff within 10s): 0.8337662337662337
Train Accuracy (abs diff within 60s): 0.8727272727272727
Non-zero Train Accuracy (abs diff within 5s): 0.3782051282051282
Non-zero Train Accuracy (abs diff within 10s): 0.38461538461538464
Non-zero Train Accuracy (abs diff within 60s): 0.44871794871794873
Test Loss: 0.09491899609565735
Test Accuracy (abs diff within 5s): 0.8072916666666666
Test Accuracy (abs diff within 10s): 0.8177083333333334
Test Accuracy (abs diff within 60s): 0.828125
Non-zero Test Accuracy (abs diff within 5s): 0.7543859649122807
Non-zero Test Accuracy (abs diff within 10s): 0.7543859649122807
Non-zero Test Accuracy (abs diff within 60s): 0.7543859649122807


### Experiment 4.2: Different activation function (sigmoid) for experiment 3
- Input:
    - Take sin and cos on: Day of the Week, Day of the Month, Day of the Year, Hour, Minute, Date, Month
    - Binary: is Weekend, is Winter Holiday
- Activation Function:
    - LSTM: tanh
    - Dense: **sigmoid**
- Loss: 
    - Mean Squared Error
- Optimizer: 
    - Adam
- Learning_rate: 
    - 0.001
- Num_epoch: 
    - 400

In [57]:
n_steps = 5
X_train, y_train, X_test, y_test, scaler, start, end = get_dataset(df_1, n_steps, start, end, 5)

model, history = train_model_sig(X_train, y_train, n_steps=n_steps)
train_pred = scaler.inverse_transform(model.predict(X_train, verbose=0))
test_pred = scaler.inverse_transform(model.predict(X_test, verbose=0))

In [58]:
plot_prediction(df_1, train_pred, test_pred, n_steps, start, end)

In [59]:
plot_loss(history)

In [60]:
evaluate(model, X_train, y_train, X_test, y_test, train_pred, test_pred)

Train Loss: 0.01874357834458351
Train Accuracy (abs diff within 5s): 0.5285714285714286
Train Accuracy (abs diff within 10s): 0.5896103896103896
Train Accuracy (abs diff within 60s): 0.7038961038961039
Non-zero Train Accuracy (abs diff within 5s): 0.14743589743589744
Non-zero Train Accuracy (abs diff within 10s): 0.19230769230769232
Non-zero Train Accuracy (abs diff within 60s): 0.3141025641025641
Test Loss: 0.04781542718410492
Test Accuracy (abs diff within 5s): 0.6354166666666666
Test Accuracy (abs diff within 10s): 0.71875
Test Accuracy (abs diff within 60s): 0.8229166666666666
Non-zero Test Accuracy (abs diff within 5s): 0.38596491228070173
Non-zero Test Accuracy (abs diff within 10s): 0.45614035087719296
Non-zero Test Accuracy (abs diff within 60s): 0.5964912280701754
